<a href="https://colab.research.google.com/github/Sami-Talab/Repos/blob/main/IPTaxApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Features: Tax Information

Simple python program that uses the user's IP address to locate/validate the tax rates and VAT numbers.

The project uses three APIs:
1. Two in conjunction, namely, IPify (used to get the user's IP address automatically) and IPstack (to get the location details).
2. Tax Data Api, to get tax rates for US and global location and validate VAT numbers.

## Steps:
1. User selects one of the three options: get tax rate, validate tax number, exit.
2. If tax rate is selected the IP address APIs get the user's IP and pass it to the Tax Api to get the tax information.
3. If validate tax number is chosen, the user inputs a location, and inputs a Tax/Vat number. Tax API will then validate it.
4. If exit is picked, the program is terminated.

## Setting Up the APIs:
STEP 1: Sign up for IPstack Api: https://ipstack.com/

*   Create an account and validate
*   Obtain the API key

STEP 2: Sign up for Tax Data Api: https://apilayer.com/marketplace/tax_data-api

*   Same procedures as step 1.

The APIs keys are stored in the Secrets tab. The function below retrieves them and it is called within the main function.

In [ ]:
from google.colab import userdata

# function to retrieve API keys
def get_api_keys():
    #API key for IPstack
    ipstack_api_key = userdata.get('IPSTACK_API_KEY')
    #API key for Tax Data
    tax_data_api_key = userdata.get('TAX_DATA_API_KEY')
    #error handling
    if not ipstack_api_key or not tax_data_api_key:
        print("API keys are missing. Please set them as secrets in the 'Secrets' tab.")
        return None, None

    return ipstack_api_key, tax_data_api_key

## Get the User's IP Address:
This function makes a simple API call to the IPify service to retrieve the user's public IP address in JSON format. It uses the endpoint https://api.ipify.org?format=json, which returns the IP address associated with the device making the request. Error handling is incorporated to manage network issues, invalid responses, or server errors.

In [ ]:
import requests

# function to get the user's public IP address
# error handling for network issues and invalid responses is included
def get_public_ip():
    try:
        response = requests.get("https://api.ipify.org?format=json")
        if response.status_code == 200:
            ip_data = response.json()
            return ip_data["ip"]
        else:
            # handle non-200 status codes, (API rate limits or server issues)
            print(f"Error: Could not retrieve public IP (status code {response.status_code})")
            return None
    except Exception as e:
        # catch and display any network errors or exceptions
        print(f"Error: {e}")
        return None

After obtaining the JSON response, this function uses the IPstack API to extract detailed location information based on the provided IP address. The endpoint http://api.ipstack.com/{ip_address} is used, where the response includes geographic details such as country, region, city, and ZIP code. The function also handles potential errors, such as API rate limits, invalid API keys, and any exceptions that may occur during the request, ensuring that any issues are logged and managed appropriately.

In [ ]:
# function to get location information from IP address
# using IPstack API, with error handling for response codes
def get_location_from_ip(ip_address, ipstack_api_key):
    url = f"http://api.ipstack.com/{ip_address}?access_key={ipstack_api_key}"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            # handle  API rate limits or invalid keys
            print(f"Error: Failed to get location data (status code {response.status_code})")
            return None
    except Exception as e:
        # any exceptions that may occur during the request
        print(f"Error: {e}")
        return None

## Getting the TAX information:
### 1. US Tax rates:

Tax API has different endpoints for US and global locations; hence the code is split into two functions.

After obtaining the JSON response from the Tax Data API, this function extracts the tax rates for a given U.S. state, including state, county, and city tax rates. The endpoint https://api.apilayer.com/tax_data/us_rate_list?state={state_code} is used, providing detailed tax information for the state. If the API call is successful, the tax data is returned, which can be used for further calculations. The code handles and ensures cases like missing tax data for the state, rate limits, or server issues are managed.


In [ ]:
# function to get US tax rates by state
# using the Tax Data API, includes error handling and status code checks
def get_us_tax_rates(state_code, tax_data_api_key):
    tax_api_url = f"https://api.apilayer.com/tax_data/us_rate_list?state={state_code}"

    headers = {
        "apikey": tax_data_api_key
    }

    try:
        response = requests.get(tax_api_url, headers=headers)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            # deal with cases where tax data is not available for the state
            print(f"Error: Tax data not found for state {state_code}. (status code 404)")
            return None
        else:
            # other potential API errors (rate limits or server issues)
            print(f"Error: Failed to get tax data (status code {response.status_code})")
            return None
    except Exception as e:
        # other  exceptions (such as network or server errors)
        print(f"Error: {e}")
        return None

## Global Tax rates:
Similarly, after receiving a successful response, this function uses the Tax Data API to retrieve global tax rates for non-U.S. locations based on the provided country code. The endpoint is different (https://api.apilayer.com/tax_data/rates?country_code={country_code}). It is used to return relevant tax information, including the standard tax rate for the specified country.

In [ ]:
# function to get global tax rates for non-US locations
# using the Tax Data API based on country code
def get_global_tax_info(country_code, tax_data_api_key):
    tax_api_url = f"https://api.apilayer.com/tax_data/rates?country_code={country_code}"

    headers = {
        "apikey": tax_data_api_key
    }

    try:
        response = requests.get(tax_api_url, headers=headers)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            # deal with cases where tax data is not found for the provided country
            print(f"Error: Tax data not found for country {country_code}. (status code 404)")
            return None
        else:
            # deal with errors like rate limits, server errors, or invalid API keys
            print(f"Error: Failed to get global tax data (status code {response.status_code})")
            return None
    except Exception as e:
        # catch other exceptions during the request
        print(f"Error: {e}")
        return None

## 3. Tax Number Validation:
This functionality was added to fully utilize the Tax Data API. It doesn't rely on the user's IP address as they are prompted to enter a country code (e.g AT for Austria) and then a Tax number.

Following the successful API request, the function validates the given VAT number for a specific country. The endpoint https://api.apilayer.com/tax_data/validate?vat_number={vat_number}&country_code={country_code} is called to confirm if the VAT number is valid.
If the response is successful, the function returns the validation details.

Error handling accounts for cases where the VAT number cannot be found, potential API rate limits, or other network issues, ensuring that any errors are captured and communicated to the user.



In [ ]:
# function to validate VAT number using the Tax Data API
def validate_tax_number(vat_number, country_code, tax_data_api_key):
    tax_api_url = f"https://api.apilayer.com/tax_data/validate?vat_number={vat_number}&country_code={country_code}"

    headers = {
        "apikey": tax_data_api_key
    }

    try:
        response = requests.get(tax_api_url, headers=headers)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            # for cases where VAT data is not found
            print(f"Error: VAT data not found for VAT number {vat_number} in country {country_code}. (status code 404)")
            return None
        else:
            # for other API errors like rate limits or invalid requests
            print(f"Error: Failed to get VAT data (status code {response.status_code})")
            return None
    except Exception as e:
        # network issues or other exceptions
        print(f"Error: {e}")
        return None

## Tax Calculation:

After performing the initial calculations, the function computes the sales tax and the total price for a given product.
The tax amount is calculated as a percentage of the product price based on the provided tax rate.
This function ensures that the final price is formatted correctly as both the sales tax and total prices are rounded to two decimal places for precision in financial transactions.

In [ ]:
# function to perform tax-compliant price calculation
def calculate_sales_tax(price, tax_rate):
    tax_amount = price * (tax_rate / 100)
    total_price = price + tax_amount

    # round both tax_amount and total_price to two decimal places
    tax_amount = round(tax_amount, 2)
    total_price = round(total_price, 2)
    return tax_amount, total_price

## Display Menu:
This function displays a simple text-based menu to the user, providing options such as retrieving tax rates, validating a tax number, or exiting the program.
The user's choice is registered and returned for subsequent processing in the main.

In [ ]:
# function to show the main menu
def show_menu():
    print("\nPlease choose an option:")
    print("1. Get Tax Rates")
    print("2. Validate Tax Number")
    print("3. Exit")
    choice = input("Enter your choice (1-3): ")
    return choice


## Main Function:
The main function ties everything together. It is responsible for the following:  retrieving API keys, displaying a menu, interacting with external APIs for tax rate and VAT number validation, handling different options the user selects from the menu as well as repeatedly showing the menu until the user chooses to exit the program.


In [ ]:
def main():
    #retrieves the API keys
    ipstack_api_key, tax_data_api_key = get_api_keys()

    #if not detected return
    if not ipstack_api_key or not tax_data_api_key:
        print("Cannot proceed without API keys.")
        return

    #loop until user decides to exit
    while True:
        choice = show_menu()

        if choice == '1':  # get Tax Rates
            # automatically get the user's public IP address
            ip_address = get_public_ip()
            if not ip_address:
                # if public IP address couldn't be retrieved, display an error and continue
                print("Error: Could not retrieve public IP address.")
                continue

            # get location data from IPstack
            location_data = get_location_from_ip(ip_address, ipstack_api_key)

            if location_data:
                # extract country code and state code from location data
                country_code = location_data.get("country_code")
                state_code = location_data.get("region_code")
                city = location_data.get("city")
                zip_code = location_data.get("zip")

                if country_code == "US" and state_code:
                    # US-specific tax rates
                    tax_data = get_us_tax_rates(state_code, tax_data_api_key)
                    if tax_data:
                        # obtain the combined tax rate and calculate sales tax
                        combined_rate = tax_data["data"][0].get("combined_rate", 0) * 100
                        print(f"Tax Data for ZIP: {zip_code}:")
                        print(f"State Rate: {tax_data['data'][0].get('state_rate', 0) * 100}%")
                        print(f"County Rate: {tax_data['data'][0].get('county_rate', 0) * 100}%")
                        print(f"City Rate: {tax_data['data'][0].get('city_rate', 0) * 100}%")
                        print(f"Combined Rate: {combined_rate}%")
                        print(f"Freight Taxable: {'Yes' if tax_data['data'][0].get('freight_taxable') else 'No'}")

                        # user input for the product price to calculate sales tax
                        price = float(input("Enter the product price: "))
                        tax_amount, total_price = calculate_sales_tax(price, combined_rate)
                        print(f"Sales tax amount: {tax_amount}")
                        print(f"Total price (including tax): {total_price}")
                    else:
                        print(f"No tax data available for state {state_code}.")
                else:
                    # global tax rates for non-US locations
                    tax_data = get_global_tax_info(country_code, tax_data_api_key)
                    if tax_data:
                        combined_rate = tax_data["standard_rate"]
                        print(f"Combined sales tax rate for {city}, {country_code}: {combined_rate}%")

                        # user input for the product price to calculate sales tax
                        price = float(input("Enter the product price: "))
                        tax_amount, total_price = calculate_sales_tax(price, combined_rate)
                        print(f"Sales tax amount: {tax_amount}")
                        print(f"Total price (including tax): {total_price}")
                    else:
                        print(f"No tax data available for country {country_code}.")
            else:
                print("Could not retrieve location information.")

        elif choice == '2':  # tax number validation
            # user input for the country code and VAT number
            country_code = input("Enter the country code (e.g., 'AT' for Austria): ")
            vat_number = input(f"Enter the VAT number for {country_code}: ")

            # VAT number validation using the Tax Data API
            vat_data = validate_tax_number(vat_number, country_code, tax_data_api_key)
            if vat_data and vat_data.get("valid"):
                vat_rate = vat_data["format_valid"]
                print(f"VAT number is valid for {vat_data['name']} in {vat_data['address']}.")
                print(f"Combined VAT rate: {vat_rate}%")
            else:
                print(f"Invalid or unavailable VAT data for VAT number {vat_number}.")

        elif choice == '3':  # termination
            print("Exiting program")
            break

        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


## Entire Code:
All the functions are collated on the code below.

In [1]:
import requests
from google.colab import userdata

# function to retrieve API keys
def get_api_keys():
    #API key for IPstack
    ipstack_api_key = userdata.get('IPSTACK_API_KEY')
    #API key for Tax Data
    tax_data_api_key = userdata.get('TAX_DATA_API_KEY')
    #error handling
    if not ipstack_api_key or not tax_data_api_key:
        print("API keys are missing. Please set them as secrets in the 'Secrets' tab.")
        return None, None

    return ipstack_api_key, tax_data_api_key

# function to get the user's public IP address
# error handling for network issues and invalid responses is included
def get_public_ip():
    try:
        response = requests.get("https://api.ipify.org?format=json")
        if response.status_code == 200:
            ip_data = response.json()
            return ip_data["ip"]
        else:
            # handle non-200 status codes, (API rate limits or server issues)
            print(f"Error: Could not retrieve public IP (status code {response.status_code})")
            return None
    except Exception as e:
        # catch and display any network errors or exceptions
        print(f"Error: {e}")
        return None

# function to get location information from IP address
# using IPstack API, with error handling for response codes
def get_location_from_ip(ip_address, ipstack_api_key):
    url = f"http://api.ipstack.com/{ip_address}?access_key={ipstack_api_key}"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            # handle  API rate limits or invalid keys
            print(f"Error: Failed to get location data (status code {response.status_code})")
            return None
    except Exception as e:
        # any exceptions that may occur during the request
        print(f"Error: {e}")
        return None

# function to get US tax rates by state
# using the Tax Data API, includes error handling and status code checks
def get_us_tax_rates(state_code, tax_data_api_key):
    tax_api_url = f"https://api.apilayer.com/tax_data/us_rate_list?state={state_code}"

    headers = {
        "apikey": tax_data_api_key
    }

    try:
        response = requests.get(tax_api_url, headers=headers)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            # deal with cases where tax data is not available for the state
            print(f"Error: Tax data not found for state {state_code}. (status code 404)")
            return None
        else:
            # other potential API errors (rate limits or server issues)
            print(f"Error: Failed to get tax data (status code {response.status_code})")
            return None
    except Exception as e:
        # other  exceptions (such as network or server errors)
        print(f"Error: {e}")
        return None

# function to get global tax rates for non-US locations
# using the Tax Data API based on country code
def get_global_tax_info(country_code, tax_data_api_key):
    tax_api_url = f"https://api.apilayer.com/tax_data/rates?country_code={country_code}"

    headers = {
        "apikey": tax_data_api_key
    }

    try:
        response = requests.get(tax_api_url, headers=headers)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            # deal with cases where tax data is not found for the provided country
            print(f"Error: Tax data not found for country {country_code}. (status code 404)")
            return None
        else:
            # deal with errors like rate limits, server errors, or invalid API keys
            print(f"Error: Failed to get global tax data (status code {response.status_code})")
            return None
    except Exception as e:
        # catch other exceptions during the request
        print(f"Error: {e}")
        return None

# function to validate VAT number using the Tax Data API
def validate_tax_number(vat_number, country_code, tax_data_api_key):
    tax_api_url = f"https://api.apilayer.com/tax_data/validate?vat_number={vat_number}&country_code={country_code}"

    headers = {
        "apikey": tax_data_api_key
    }

    try:
        response = requests.get(tax_api_url, headers=headers)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            # for cases where VAT data is not found
            print(f"Error: VAT data not found for VAT number {vat_number} in country {country_code}. (status code 404)")
            return None
        else:
            # for other API errors like rate limits or invalid requests
            print(f"Error: Failed to get VAT data (status code {response.status_code})")
            return None
    except Exception as e:
        # network issues or other exceptions
        print(f"Error: {e}")
        return None

def calculate_sales_tax(price, tax_rate):
    tax_amount = price * (tax_rate / 100)
    total_price = price + tax_amount

    # round both tax_amount and total_price to two decimal places
    tax_amount = round(tax_amount, 2)
    total_price = round(total_price, 2)
    return tax_amount, total_price

# function to show the main menu
def show_menu():
    print("\nPlease choose an option:")
    print("1. Get Tax Rates")
    print("2. Validate Tax Number")
    print("3. Exit")
    choice = input("Enter your choice (1-3): ")
    return choice

def main():
    #retrieves the API keys
    ipstack_api_key, tax_data_api_key = get_api_keys()

    #if not detected return
    if not ipstack_api_key or not tax_data_api_key:
        print("Cannot proceed without API keys.")
        return

    #loop until user decides to exit
    while True:
        choice = show_menu()

        if choice == '1':  # get Tax Rates
            # automatically get the user's public IP address
            ip_address = get_public_ip()
            if not ip_address:
                # if public IP address couldn't be retrieved, display an error and continue
                print("Error: Could not retrieve public IP address.")
                continue

            # get location data from IPstack
            location_data = get_location_from_ip(ip_address, ipstack_api_key)

            if location_data:
                # extract country code and state code from location data
                country_code = location_data.get("country_code")
                state_code = location_data.get("region_code")
                city = location_data.get("city")
                zip_code = location_data.get("zip")

                if country_code == "US" and state_code:
                    # US-specific tax rates
                    tax_data = get_us_tax_rates(state_code, tax_data_api_key)
                    if tax_data:
                        # obtain the combined tax rate and calculate sales tax
                        combined_rate = tax_data["data"][0].get("combined_rate", 0) * 100
                        print(f"Tax Data for ZIP: {zip_code}:")
                        print(f"State Rate: {tax_data['data'][0].get('state_rate', 0) * 100}%")
                        print(f"County Rate: {tax_data['data'][0].get('county_rate', 0) * 100}%")
                        print(f"City Rate: {tax_data['data'][0].get('city_rate', 0) * 100}%")
                        print(f"Combined Rate: {combined_rate}%")
                        print(f"Freight Taxable: {'Yes' if tax_data['data'][0].get('freight_taxable') else 'No'}")

                        # user input for the product price to calculate sales tax
                        price = float(input("Enter the product price: "))
                        tax_amount, total_price = calculate_sales_tax(price, combined_rate)
                        print(f"Sales tax amount: {tax_amount}")
                        print(f"Total price (including tax): {total_price}")
                    else:
                        print(f"No tax data available for state {state_code}.")
                else:
                    # global tax rates for non-US locations
                    tax_data = get_global_tax_info(country_code, tax_data_api_key)
                    if tax_data:
                        combined_rate = tax_data["standard_rate"]
                        print(f"Combined sales tax rate for {city}, {country_code}: {combined_rate}%")

                        # user input for the product price to calculate sales tax
                        price = float(input("Enter the product price: "))
                        tax_amount, total_price = calculate_sales_tax(price, combined_rate)
                        print(f"Sales tax amount: {tax_amount}")
                        print(f"Total price (including tax): {total_price}")
                    else:
                        print(f"No tax data available for country {country_code}.")
            else:
                print("Could not retrieve location information.")

        elif choice == '2':  # tax number validation
            # user input for the country code and VAT number
            country_code = input("Enter the country code (e.g., 'AT' for Austria): ")
            vat_number = input(f"Enter the VAT number for {country_code}: ")

            # VAT number validation using the Tax Data API
            vat_data = validate_tax_number(vat_number, country_code, tax_data_api_key)
            if vat_data and vat_data.get("valid"):
                vat_rate = vat_data["format_valid"]
                print(f"VAT number is valid for {vat_data['name']} in {vat_data['address']}.")
                print(f"Combined VAT rate: {vat_rate}%")
            else:
                print(f"Invalid or unavailable VAT data for VAT number {vat_number}.")

        elif choice == '3':  # termination
            print("Exiting program")
            break

        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()



Please choose an option:
1. Get Tax Rates
2. Validate Tax Number
3. Exit
Enter your choice (1-3): 1
Tax Data for ZIP: 51501:
State Rate: 6.0%
County Rate: 1.0%
City Rate: 0%
Combined Rate: 7.000000000000001%
Freight Taxable: No
Enter the product price: 34
Sales tax amount: 2.38
Total price (including tax): 36.38

Please choose an option:
1. Get Tax Rates
2. Validate Tax Number
3. Exit
Enter your choice (1-3): 3
Exiting program
